## Import Packages

In [1]:
from fastbook import *
from fastai.vision.widgets import *
import torch
from fastai.tabular.all import *

## Constants

In [2]:
# Set all constants
training_data_path = "Images"
model_path = "Model/"
img_size = 128

## Data Loading & Augmentation

In [3]:
model_data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(img_size)
)

In [4]:
dls = model_data.dataloaders(training_data_path)

In [5]:
model_aug = model_data.new(item_tfms=Resize(img_size), batch_tfms=aug_transforms(mult=2))
dls = model_aug.dataloaders(training_data_path)
dls.train.show_batch(max_n=4, nrows=2, unique=True)

Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


NotImplementedError: The operator 'aten::_linalg_solve_ex.result' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

## Model Training

In [2]:
learn = vision_learner(dls, resnet34, metrics=[accuracy, error_rate])

In [ ]:
learning_rate = learn.lr_find()
learning_rate

In [ ]:
learn.fit(3, lr=learning_rate)

## Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=5)

In [ ]:
interp.print_classification_report()

In [ ]:
learn.export(model_path + "Model.pkl")
torch.save(learn, model_path + "Pt_Model.pt")

## Convert the Model to ONNX Model

In [ ]:
import torch
import torch.onnx

learn = load_learner(model_path + 'Model2.pkl')
model = learn.model.eval()
dummy_data = torch.randn(1, 3, img_size, img_size)
dummy_input = torch.autograd.Variable(dummy_data)

output_model_name = 'onnx_greenscape2.onnx'
input_name = 'image'
output_name = 'warn'

torch.onnx.export(model, dummy_input, model_path + output_model_name, input_names = [input_name], output_names = [output_name])

In [ ]:
import onnx
onnx_model = onnx.load(model_path + output_model_name)
onnx.checker.check_model(onnx_model)
model